# AutoKeras

* https://autokeras.com/

## Imports and Global Settings

In [1]:
import sys
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split

sys.path.append('../')
from passkeys import RDS_ENDPOINT, RDS_PASSWORD

# Pandas Settings
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.options.display.max_info_columns = 200
pd.options.display.precision = 5

## Database Connection

In [2]:
username = 'postgres'
password = RDS_PASSWORD
endpoint = RDS_ENDPOINT
database = 'nba_betting'
port = '5432'

connection = create_engine(f'postgresql+psycopg2://{username}:{password}@{endpoint}/{database}').connect()

### Datasets

In [3]:
df = pd.read_sql_table('nba_model_ready', connection)

## Basic Data Overview

In [4]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9185 entries, 0 to 9184
Data columns (total 1219 columns):
 #     Column                                 Non-Null Count  Dtype  
---    ------                                 --------------  -----  
 0     game_id                                9185 non-null   object 
 1     CLS_TARGET_home_margin_GT_home_spread  9185 non-null   bool   
 2     REG_TARGET_actual_home_margin          9185 non-null   float64
 3     home_team_num                          9185 non-null   int64  
 4     away_team_num                          9185 non-null   int64  
 5     league_year_end                        9185 non-null   int64  
 6     fd_line_home                           0 non-null      float64
 7     dk_line_home                           0 non-null      float64
 8     covers_consenses_home                  0 non-null      float64
 9     home_spread                            9185 non-null   float64
 10    gp                                     9157 

In [5]:
df.head()

,game_id,CLS_TARGET_home_margin_GT_home_spread,REG_TARGET_actual_home_margin,home_team_num,away_team_num,league_year_end,fd_line_home,dk_line_home,covers_consenses_home,home_spread,gp,win,loss,w_pct,mins,pts,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,tov,stl,blk,blka,pf,pfd,p_m,gp_rank,gp_vla,gp_vla_std,win_rank,win_vla,win_vla_std,loss_rank,loss_vla,loss_vla_std,w_pct_rank,w_pct_vla,w_pct_vla_std,mins_rank,mins_vla,mins_vla_std,pts_rank,pts_vla,pts_vla_std,fgm_rank,fgm_vla,fgm_vla_std,fga_rank,fga_vla,fga_vla_std,fg_pct_rank,fg_pct_vla,fg_pct_vla_std,fg3m_rank,fg3m_vla,fg3m_vla_std,fg3a_rank,fg3a_vla,fg3a_vla_std,fg3_pct_rank,fg3_pct_vla,fg3_pct_vla_std,ftm_rank,ftm_vla,ftm_vla_std,fta_rank,fta_vla,fta_vla_std,ft_pct_rank,ft_pct_vla,ft_pct_vla_std,oreb_rank,oreb_vla,oreb_vla_std,dreb_rank,dreb_vla,dreb_vla_std,reb_rank,reb_vla,reb_vla_std,ast_rank,ast_vla,ast_vla_std,tov_rank,tov_vla,tov_vla_std,stl_rank,stl_vla,stl_vla_std,blk_rank,blk_vla,blk_vla_std,blka_rank,blka_vla,blka_vla_std,pf_rank,pf_vla,pf_vla_std,pfd_rank,pfd_vla,pfd_vla_std,p_m_rank,p_m_vla,p_m_vla_std,gp_opp,win_opp,loss_opp,w_pct_opp,mins_opp,pts_opp,fgm_opp,fga_opp,fg_pct_opp,fg3m_opp,fg3a_opp,fg3_pct_opp,ftm_opp,fta_opp,ft_pct_opp,oreb_opp,dreb_opp,reb_opp,ast_opp,tov_opp,stl_opp,blk_opp,blka_opp,pf_opp,pfd_opp,p_m_opp,gp_rank_opp,gp_vla_opp,gp_vla_std_opp,win_rank_opp,win_vla_opp,win_vla_std_opp,loss_rank_opp,loss_vla_opp,loss_vla_std_opp,w_pct_rank_opp,w_pct_vla_opp,w_pct_vla_std_opp,mins_rank_opp,mins_vla_opp,mins_vla_std_opp,pts_rank_opp,pts_vla_opp,pts_vla_std_opp,fgm_rank_opp,fgm_vla_opp,fgm_vla_std_opp,fga_rank_opp,fga_vla_opp,fga_vla_std_opp,fg_pct_rank_opp,fg_pct_vla_opp,fg_pct_vla_std_opp,fg3m_rank_opp,fg3m_vla_opp,fg3m_vla_std_opp,fg3a_rank_opp,fg3a_vla_opp,fg3a_vla_std_opp,fg3_pct_rank_opp,fg3_pct_vla_opp,fg3_pct_vla_std_opp,ftm_rank_opp,ftm_vla_opp,ftm_vla_std_opp,fta_rank_opp,fta_vla_opp,fta_vla_std_opp,ft_pct_rank_opp,ft_pct_vla_opp,ft_pct_vla_std_opp,oreb_rank_opp,oreb_vla_opp,oreb_vla_std_opp,dreb_rank_opp,dreb_vla_opp,dreb_vla_std_opp,reb_rank_opp,reb_vla_opp,reb_vla_std_opp,ast_rank_opp,ast_vla_opp,ast_vla_std_opp,tov_rank_opp,tov_vla_opp,tov_vla_std_opp,stl_rank_opp,stl_vla_opp,stl_vla_std_opp,blk_rank_opp,blk_vla_opp,blk_vla_std_opp,blka_rank_opp,blka_vla_opp,blka_vla_std_opp,pf_rank_opp,pf_vla_opp,pf_vla_std_opp,pfd_rank_opp,pfd_vla_opp,pfd_vla_std_opp,p_m_rank_opp,p_m_vla_opp,p_m_vla_std_opp,offrtg,defrtg,netrtg,ast_pct,ast_v_tov,ast_ratio,oreb_pct,dreb_pct,reb_pct,tov_pct,efg_pct,ts_pct,pace,pie,poss,offrtg_rank,offrtg_vla,offrtg_vla_std,defrtg_rank,defrtg_vla,defrtg_vla_std,netrtg_rank,netrtg_vla,netrtg_vla_std,ast_pct_rank,ast_pct_vla,ast_pct_vla_std,ast_v_tov_rank,ast_v_tov_vla,ast_v_tov_vla_std,ast_ratio_rank,ast_ratio_vla,...,opp_fgm_c3,opp_fga_c3,opp_fg_pct_c3,opp_fgm_atb3,opp_fga_atb3,opp_fg_pct_atb3,opp_fgm_ra_rank,opp_fgm_ra_vla,opp_fgm_ra_vla_std,opp_fga_ra_rank,opp_fga_ra_vla,opp_fga_ra_vla_std,opp_fg_pct_ra_rank,opp_fg_pct_ra_vla,opp_fg_pct_ra_vla_std,opp_fgm_paint_rank,opp_fgm_paint_vla,opp_fgm_paint_vla_std,opp_fga_paint_rank,opp_fga_paint_vla,opp_fga_paint_vla_std,opp_fg_pct_paint_rank,opp_fg_pct_paint_vla,opp_fg_pct_paint_vla_std,opp_fgm_mr_rank,opp_fgm_mr_vla,opp_fgm_mr_vla_std,opp_fga_mr_rank,opp_fga_mr_vla,opp_fga_mr_vla_std,opp_fg_pct_mr_rank,opp_fg_pct_mr_vla,opp_fg_pct_mr_vla_std,opp_fgm_lc3_rank,opp_fgm_lc3_vla,opp_fgm_lc3_vla_std,opp_fga_lc3_rank,opp_fga_lc3_vla,opp_fga_lc3_vla_std,opp_fg_pct_lc3_rank,opp_fg_pct_lc3_vla,opp_fg_pct_lc3_vla_std,opp_fgm_rc3_rank,opp_fgm_rc3_vla,opp_fgm_rc3_vla_std,opp_fga_rc3_rank,opp_fga_rc3_vla,opp_fga_rc3_vla_std,opp_fg_pct_rc3_rank,opp_fg_pct_rc3_vla,opp_fg_pct_rc3_vla_std,opp_fgm_c3_rank,opp_fgm_c3_vla,opp_fgm_c3_vla_std,opp_fga_c3_rank,opp_fga_c3_vla,opp_fga_c3_vla_std,opp_fg_pct_c3_rank,opp_fg_pct_c3_vla,opp_fg_pct_c3_vla_std,opp_fgm_atb3_rank,opp_fgm_atb3_vla,opp_fgm_atb3_vla_std,opp_fga_atb3_rank,opp_fga_atb3_vla,opp_fga_atb3_vla_std,opp_fg_pct_atb3

## AutoKeras - Regression

* https://autokeras.com/tutorial/structured_data_regression/

### Setup and Preprocessing

In [6]:
features_to_drop = ['game_id', 'CLS_TARGET_home_margin_GT_home_spread']
keep_features = ['REG_TARGET_actual_home_margin',
                 'home_team_num', 'away_team_num',
                 'league_year_end', 'home_spread']
features_to_use = [feature for feature in list(df) if feature[-3:] == 'vla'] + keep_features

In [7]:
reg_df = df.drop(columns=features_to_drop)
reg_df = reg_df[features_to_use]

In [8]:
reg_df.head()

,gp_vla,win_vla,loss_vla,w_pct_vla,mins_vla,pts_vla,fgm_vla,fga_vla,fg_pct_vla,fg3m_vla,fg3a_vla,fg3_pct_vla,ftm_vla,fta_vla,ft_pct_vla,oreb_vla,dreb_vla,reb_vla,ast_vla,tov_vla,stl_vla,blk_vla,blka_vla,pf_vla,pfd_vla,p_m_vla,offrtg_vla,defrtg_vla,netrtg_vla,ast_pct_vla,ast_v_tov_vla,ast_ratio_vla,oreb_pct_vla,dreb_pct_vla,reb_pct_vla,tov_pct_vla,efg_pct_vla,ts_pct_vla,pace_vla,pie_vla,poss_vla,fta_rate_vla,opp_efg_pct_vla,opp_fta_rate_vla,opp_tov_pct_vla,opp_oreb_pct_vla,pts_off_tov_vla,second_pts_vla,fbps_vla,pitp_vla,opp_pts_off_tov_vla,opp_second_pts_vla,opp_fbps_vla,opp_pitp_vla,pct_fga_2pt_vla,pct_fga_3pt_vla,pct_pts_2pt_vla,pct_pts_2pt_mid_vla,pct_pts_3pt_vla,pct_pts_fbps_vla,pct_pts_ft_vla,pct_pts_off_tov_vla,pct_pts_pitp_vla,pct_ast_2fgm_vla,pct_uast_2fgm_vla,pct_ast_3fgm_vla,pct_uast_3fgm_vla,pct_ast_fgm_vla,pct_uast_fgm_vla,opp_fgm_vla,opp_fga_vla,opp_fg_pct_vla,opp_3pm_vla,opp_3pa_vla,opp_3pt_pct_vla,opp_ftm_vla,opp_fta_vla,opp_ft_pct_vla,opp_oreb_vla,opp_dreb_vla,opp_reb_vla,opp_ast_vla,opp_tov_vla,opp_stl_vla,opp_blk_vla,opp_blka_vla,opp_pf_vla,opp_pfd_vla,opp_pts_vla,opp_pm_vla,dist_feet_vla,dist_miles_vla,dist_miles_off_vla,dist_miles_def_vla,avg_speed_vla,avg_speed_off_vla,avg_speed_def_vla,fgm_ra_vla,fga_ra_vla,fg_pct_ra_vla,fgm_paint_vla,fga_paint_vla,fg_pct_paint_vla,fgm_mr_vla,fga_mr_vla,fg_pct_mr_vla,fgm_lc3_vla,fga_lc3_vla,fg_pct_lc3_vla,fgm_rc3_vla,fga_rc3_vla,fg_pct_rc3_vla,fgm_c3_vla,fga_c3_vla,fg_pct_c3_vla,fgm_atb3_vla,fga_atb3_vla,fg_pct_atb3_vla,opp_fgm_ra_vla,opp_fga_ra_vla,opp_fg_pct_ra_vla,opp_fgm_paint_vla,opp_fga_paint_vla,opp_fg_pct_paint_vla,opp_fgm_mr_vla,opp_fga_mr_vla,opp_fg_pct_mr_vla,opp_fgm_lc3_vla,opp_fga_lc3_vla,opp_fg_pct_lc3_vla,opp_fgm_rc3_vla,opp_fga_rc3_vla,opp_fg_pct_rc3_vla,opp_fgm_c3_vla,opp_fga_c3_vla,opp_fg_pct_c3_vla,opp_fgm_atb3_vla,opp_fga_atb3_vla,opp_fg_pct_atb3_vla,screen_ast_vla,screen_ast_pts_vla,deflections_vla,off_loose_ball_rec_vla,def_loose_ball_rec_vla,loose_ball_rec_vla,pct_loose_ball_rec_off_vla,pct_loose_ball_rec_def_vla,charges_drawn_vla,contested_2pt_vla,contested_3pt_vla,contested_shots_vla,REG_TARGET_actual_home_margin,home_team_num,away_team_num,league_year_end,home_spread
0,-0.13333,-17.06667,16.93333,-0.21223,0.02667,-5.61000,-2.38333,0.59333,-2.96333,-1.13000,-0.69667,-2.53000,0.28667,0.05000,0.83667,0.65000,-2.06000,-1.48667,-1.20000,0.33000,0.16333,0.06000,0.56000,2.35333,0.15667,-7.40333,-5.59000,1.72667,-7.42,0.59000,-0.13533,-0.81667,0.05333,-1.91667,-1.91333,0.34333,-3.63000,-3.25333,0.03967,-4.89667,-16.93333,-0.00123,0.98333,0.04480,0.94667,1.91667,0.82667,0.32667,-2.08000,-1.48333,0.52667,-0.79,-1.48333,1.81667,1.15333,-1.15333,0.36667,-0.55333,-1.45000,-1.36667,1.08667,1.58333,0.91333,-1.69333,1.69333,7.12333,-7.12333,0.59000,-0.59000,-0.08333,-2.30667,0.01140,-0.53667,-2.58667,0.01153,2.59000,3.25,-0.00010,-0.04333,1.73667,1.81000,0.48000,0.92667,-0.33000,0.56000,0.06000,0.15667,2.35333,1.79333,7.40333,1928.71030,0.36567,0.25800,0.10733,0.05867,0.07600,0.04100,0.31667,2.41667,-0.04617,-1.01667,-0.94333,-0.04047,-0.49000,-0.19333,-0.03460,0.22000,0.98000,-0.02400,-0.05667,0.08667,-0.01203,0.26000,1.05667,-0.01890,-1.27333,-1.69667,-0.02893,0.92000,1.90667,-0.01157,-0.01333,-0.73333,0.01953,-0.48667,-0.88000,-0.00920,0.22333,-0.12000,0.05327,0.13333,0.07667,0.02657,0.35333,-0.03667,0.04067,-0.86667,-2.41000,-0.00183,-1.10200,-2.94333,0.99333,0.06367,0.11167,0.17533,-0.00303,0.00303,0.12900,2.48533,-0.85233,1.63333,-30.0,9,17,22,10.5
1,0.33333,4.16667,-3.83333,0.05100,-0.18333,-2.96667,-0.94667,-3.48333,0.78000,-0.88667,-2.44667,-0.15000,-0.09667,0.19333,-1.29333,-0.06333,-0.15667,-0.21333,0.57667,0.82333,-0.52000,-0.53000,-0.23667,-2.33667,0.47000,2.30333,-0.49667,-2.67000,2.18,2.92000,-0.07000,0.72333,0.50667,0.02667,0.69667,1.13000,0.53333,0.40333,-1.96433,1.90667,-139.86667,0.01227,-1.15667,-0.02977,0.13333,-0.02667,-0.23667,-0.47667,-1.54667,1.14667,1.26000,-1.07,-0.23667,-0.45667,1.23667,-1.23667,1.30667,-0.96667,-1.59000,-1.15667,0.27

In [9]:
reg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9185 entries, 0 to 9184
Data columns (total 156 columns):
 #    Column                         Non-Null Count  Dtype  
---   ------                         --------------  -----  
 0    gp_vla                         9157 non-null   float64
 1    win_vla                        9157 non-null   float64
 2    loss_vla                       9157 non-null   float64
 3    w_pct_vla                      9157 non-null   float64
 4    mins_vla                       9157 non-null   float64
 5    pts_vla                        9157 non-null   float64
 6    fgm_vla                        9157 non-null   float64
 7    fga_vla                        9157 non-null   float64
 8    fg_pct_vla                     9157 non-null   float64
 9    fg3m_vla                       9157 non-null   float64
 10   fg3a_vla                       9157 non-null   float64
 11   fg3_pct_vla                    9157 non-null   float64
 12   ftm_vla                        9

In [10]:
def split_data(df, target, test_size=0.3, random_state=0):
    """Deep Learning specific wrapper for normal sklearn train/test split.

    Args:
        df (Pandas Dataframe)
        target (str): Name of target column.
        test_size (float, optional): Defaults to 0.3.
        random_state (int, optional): Defaults to 0.

    Returns:
        Tuple of Dataframes: X_train, X_test, y_train, y_test
    """
    y = df[target]
    X = df.drop(columns=target)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state)
    print(f"X_train Shape: {X_train.shape}")
    print(f"X_test Shape: {X_test.shape}")
    print(f"y_train Shape: {y_train.shape}")
    print(f"y_test Shape: {y_test.shape}")
    return X_train, X_test, y_train, y_test

In [11]:
cls_target = 'CLS_TARGET_home_margin_GT_home_spread'
reg_target = 'REG_TARGET_actual_home_margin'

In [12]:
X_train, X_test, y_train, y_test = split_data(reg_df, reg_target)

X_train Shape: (6429, 155)
X_test Shape: (2756, 155)
y_train Shape: (6429,)
y_test Shape: (2756,)


### Fit

In [13]:
reg = ak.StructuredDataRegressor(max_trials=20, overwrite=True)

In [14]:
reg.fit(X_train, y_train)

Trial 20 Complete [00h 00m 26s]
val_loss: 184.30673217773438

Best val_loss So Far: 182.494384765625
Total elapsed time: 00h 09m 15s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
201/201 [==============================] - 3s 4ms/step - loss: 188.7577 - mean_squared_error: 188.7577
Epoch 2/10
201/201 [==============================] - 1s 4ms/step - loss: 184.5103 - mean_squared_error: 184.5103
Epoch 3/10
201/201 [==============================] - 1s 4ms/step - loss: 182.4432 - mean_squared_error: 182.4432
Epoch 4/10
201/201 [==============================] - 1s 5ms/step - loss: 180.0244 - mean_squared_error: 180.0244
Epoch 5/10
201/201 [==============================] - 1s 5ms/step - loss: 179.3797 - mean_squared_error: 179.3797
Epoch 6/10
201/201 [==============================] - 1s 5ms/step - loss: 179.9292 - mean_squared_error: 179.9292
Epoch 7/10
201/201 [==============================] - 1s 5ms/step - loss: 178.5939 - mean_squared_error: 178.5939
Epoch 8/10
201/201 [===========

### Predict

In [15]:
predicted_y = reg.predict(X_test)

87/87 [==============================] - 0s 4ms/step


### Evaluate

In [16]:
print(reg.evaluate(X_test, y_test))

87/87 [==============================] - 1s 3ms/step - loss: 175.0214 - mean_squared_error: 175.0214
[175.0213623046875, 175.0213623046875]


### Model Finalization and Storage

In [17]:
final_reg_model = reg.export_model()

In [18]:
final_reg_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 155)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 155)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 155)              311       
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                4992      
                                                                 
 batch_normalization (BatchN  (None, 32)               128       
 ormalization)                                                   
                                                             

In [19]:
print(type(final_reg_model))

<class 'keras.engine.functional.Functional'>


In [51]:
# try:
#     final_reg_model.save('../models/AutoKeras/Baseline_AK_REG', save_format="tf")
# except Exception:
#     final_reg_model.save('../models/AutoKeras/Baseline_AK_REG.h5')

## Classification

### Setup and Preprocessing

In [21]:
features_to_drop = ['game_id', 'REG_TARGET_actual_home_margin']
keep_features = ['CLS_TARGET_home_margin_GT_home_spread',
                 'home_team_num', 'away_team_num',
                 'league_year_end', 'home_spread']
features_to_use = [feature for feature in list(df) if feature[-3:] == 'vla'] + keep_features

In [22]:
cls_df = df.drop(columns=features_to_drop)
cls_df = cls_df[features_to_use]

In [23]:
cls_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9185 entries, 0 to 9184
Data columns (total 156 columns):
 #    Column                                 Non-Null Count  Dtype  
---   ------                                 --------------  -----  
 0    gp_vla                                 9157 non-null   float64
 1    win_vla                                9157 non-null   float64
 2    loss_vla                               9157 non-null   float64
 3    w_pct_vla                              9157 non-null   float64
 4    mins_vla                               9157 non-null   float64
 5    pts_vla                                9157 non-null   float64
 6    fgm_vla                                9157 non-null   float64
 7    fga_vla                                9157 non-null   float64
 8    fg_pct_vla                             9157 non-null   float64
 9    fg3m_vla                               9157 non-null   float64
 10   fg3a_vla                               9157 non-null   flo

In [24]:
X_train, X_test, y_train, y_test = split_data(cls_df, cls_target)

X_train Shape: (6429, 155)
X_test Shape: (2756, 155)
y_train Shape: (6429,)
y_test Shape: (2756,)


### Fit

In [25]:
cls = ak.StructuredDataClassifier(max_trials=20, overwrite=True)

In [26]:
cls.fit(X_train, y_train)

Trial 20 Complete [00h 00m 22s]
val_accuracy: 0.5262333750724792

Best val_accuracy So Far: 0.5340642333030701
Total elapsed time: 00h 06m 46s
INFO:tensorflow:Oracle triggered exit
Epoch 1/11
201/201 [==============================] - 2s 4ms/step - loss: 0.7179 - accuracy: 0.5038
Epoch 2/11
201/201 [==============================] - 1s 4ms/step - loss: 0.6923 - accuracy: 0.5362
Epoch 3/11
201/201 [==============================] - 1s 4ms/step - loss: 0.6875 - accuracy: 0.5433
Epoch 4/11
201/201 [==============================] - 1s 4ms/step - loss: 0.6842 - accuracy: 0.5491
Epoch 5/11
201/201 [==============================] - 1s 4ms/step - loss: 0.6807 - accuracy: 0.5606
Epoch 6/11
201/201 [==============================] - 1s 4ms/step - loss: 0.6773 - accuracy: 0.5688
Epoch 7/11
201/201 [==============================] - 1s 4ms/step - loss: 0.6733 - accuracy: 0.5813
Epoch 8/11
201/201 [==============================] - 1s 4ms/step - loss: 0.6710 - accuracy: 0.5780
Epoch 9/11
201/201 

### Predict

In [27]:
predicted_y = cls.predict(X_test)

87/87 [==============================] - 0s 4ms/step


### Evaluate

In [28]:
print(cls.evaluate(X_test, y_test))

87/87 [==============================] - 1s 3ms/step - loss: 0.7175 - accuracy: 0.5076
[0.7174785733222961, 0.5076197385787964]


### Model Finalization and Storage

In [29]:
final_cls_model = cls.export_model()

In [30]:
final_cls_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 155)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 155)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 155)              311       
 n)                                                              
                                                                 
 dense (Dense)               (None, 256)               39936     
                                                                 
 re_lu (ReLU)                (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                16448 

In [31]:
print(type(final_cls_model))

<class 'keras.engine.functional.Functional'>


In [50]:
# try:
#     final_cls_model.save('../models/AutoKeras/Baseline_AK_CLS', save_format="tf")
# except Exception:
#     final_cls_model.save('../models/AutoKeras/Baseline_AK_CLS.h5')